In [149]:
import numpy as np
import pandas as pd
import tweepy
import sqlite3
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import API
import matplotlib.pyplot as plt
import time
import pickle
import joblib

In [150]:
#!pip install sqlite3
access_token=''
access_token_secret=''
consumer_key=''
consumer_secret=''

In [153]:
def createdatabase():
    try:
        conn=sqlite3.connect("Twitter_storage2.db")
        print("Database connection successfull")
        conn.execute('''CREATE TABLE search_twitter_data
                (Date TEXT,
                Text TEXT,
                UserName TEXT,
                ScreenName TEXT,
                Bio TEXT,
                FollowersCount INT,
                FollowingCount INT,
                Language TEXT,
                RetweetCount INT,
                Location TEXT,
                Id_str TEXT,
                Source TEXT,
                Link TEXT);''')
        print('Table created successfully')
        conn.close()
    except:
        print('table already exists')
    
    return()
    

In [154]:
createdatabase()
conn=sqlite3.connect("Twitter_storage.db")
auth=OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

#wait_on_rate_limit whether or not to automatically wait for rate limit to replenish
#wait_on_rate_limit_notify whether or not to print a notification when Tweepy is waiting for rate limi to replenish
#Sleep mode is automatically enabled with these two params

Database connection successfull
Table created successfully


In [155]:
tweetsPerQuery=100 #default
maxTweets=10000 #for while loop

#No sinceId and maxId 
sinceId=None
maxId=-1
tweet_count=0

print('Downloading tweets...takes some time')

searchquery='samsung'
searchquery1="(samsung phone) OR (samsung smartphone) OR (samsung mobile) OR iphone OR motorola\
OR xiaomi OR redmi OR oppo OR oneplus OR (google pixel) OR (realme phone) OR (realme smartphone)\
OR (realme mobile) OR (huawei phone) OR (huawei smartphone) OR (huawei mobile) OR (huawei series)\
OR (samsung galaxy) OR (samsung series) OR (samsung note) OR (google nexus) OR (POCO smartphone)\
OR (poco mobile) OR (poco phone) OR (vivo phone) OR (vivo smartphone) OR (vivo mobile)"

#Knowledge
#AND is the default e.g query="big data" is "big" and "data" in the tweet.
#OR is OR operator e.g.query="like" OR "love"
#Negation is done using "-" symbol e.g query="apple-phone"


In [157]:
data=[]
while (tweet_count<maxTweets):
    conn=sqlite3.connect("Twitter_storage.db")
    try:
        if(maxId<=0):
            if(not sinceId):
                new_tweets=api.search(q=searchquery1+"-filter:retweets",count=tweetsPerQuery,lang="en",tweet_mode="extended")
            else:
                new_tweets=api.search(q=searchquery1+"-filter:retweets",count=tweetsPerQuery,lang="en",tweet_mode="extended",sinceId=sinceId)
        else:
            if(not sinceId):
                new_tweets=api.search(q=searchquery1+"-filter:retweets",count=tweetsPerQuery,lang="en",tweet_mode="extended",maxId=str(maxId-1))
            else:
                new_tweets=api.search(q=searchquery1+"-filter:retweets",count=tweetsPerQuery,lang="en",tweet_mode="extended",maxId=str(maxId-1),sinceId=sinceId)
        
        #No Tweets 
        if(not new_tweets):
            break
        
        for tweet in new_tweets:
            date=tweet.created_at
            text=tweet.full_text            
            username=tweet.user.name            
            screenname=tweet.user.screen_name            
            bio=tweet.user.description           
            followerscount=tweet.user.followers_count
            followingcount=tweet.user.friends_count           
            language=tweet.lang
            retweetcount=tweet.retweet_count
            location=tweet.user.location            
            id_str=tweet.id_str            
            source=tweet.source            
            #print(username)
            link='https://twitter.com/'+str(''.join(screenname))+'/status/'+str(''.join(id_str))
            
            conn.execute("INSERT INTO search_twitter_data VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)",(date,text,username,screenname,bio,followerscount,followingcount,language,retweetcount,location,id_str,source,link))
            conn.commit()
            #Additional info f.write(jsonpickle.encode(tweet._json,unpicklable=False)+'\n')
            #(Date,Text,UserName,ScreenName,Bio,FollowersCount,FollowingCount,Language,RetweetCount,Location,Id_str,Source,Link)
            #(str(date),str(text),str(username),str(screenname),str(bio),int(followerscount),int(followingcount),str(language),int(retweetcount),str(location),str(id_str),str(source),str(link))
        tweet_count+=len(new_tweets)
        maxId=new_tweets[-1].id
        if tweet_count % 80000==0:
            print(tweet_count)        
        
    except tweepy.TweepError as e:
        print("Some error!!:"+str(e))
        continue
conn.close()

print("Successfully Completed")
        

Successfully Completed


In [11]:
import sqlite3
import pandas as pd

In [12]:
conn=sqlite3.connect("Twitter_storage.db")
conn.cursor()
df=pd.read_sql_query("SELECT * from search_twitter_data",conn)
conn.close()

In [13]:
df.count()

Date              1031
Text              1031
UserName          1031
ScreenName        1031
Bio               1031
FollowersCount    1031
FollowingCount    1031
Language          1031
RetweetCount      1031
Location          1031
Id_str            1031
Source            1031
Link              1031
dtype: int64

In [15]:
df.head(20)

,Date,Text,UserName,ScreenName,Bio,FollowersCount,FollowingCount,Language,RetweetCount,Location,Id_str,Source,Link
0,2020-11-12 04:01:40,RT @SuperSaf: NEW VIDEO: iPhone 12 Pro Max vs ...,planerbeat,planerbeat,ЛГБТ-инструктор,28,111,en,43,Объединенные Рабские Вымираты,1326736897792675841,Tweetbot for iΟS,https://twitter.com/planerbeat/status/13267368...
1,2020-11-12 04:00:55,"RT @SamsungMobile: Make their year, with Galax...",Victory⁷BE🧨Dynamite,Khloe716,보라해💜BTS 💜아미💜🌈\n'Teamwork Makes The Dream Work'...,58,215,en,2085,,1326736708046483459,Twitter for Android,https://twitter.com/Khloe716/status/1326736708...
2,2020-11-12 04:00:41,RT @baejinnuna: Previously Hyunsuk mention in ...,BiiBoo ~ FIX,wana_santika,#CIX,75,314,en,48,,1326736648718016512,Twitter for Android,https://twitter.com/wana_santika/status/132673...
3,2020-11-12 04:00:16,RT @yangwoning0209: #JUNGWON jungwonie bought ...,•0902•ʕ•ᴥ•ʔ,pkyjungwon,คลั่งรัก จองวอน #Enhypen\n@ENHYPEN_members\n#E...,23,79,en,69,,1326736542899924992,Twitter for Android,https://twitter.com/pkyjungwon/status/13267365...
4,2020-11-12 04:00:13,RT @baejinnuna: Previously Hyunsuk mention in ...,Mia Yales Oliverio 🌸,miaxxi0514,BYOUNGGON's GONISAUR 💙\n\nBOBBY's 💜\n\nFIX | i...,100,148,en,48,Philippines Seoul,1326736532053483520,Twitter for iPhone,https://twitter.com/miaxxi0514/status/13267365...
5,2020-11-12 03:59:42,RT @yangwoning0209: #JUNGWON jungwonie bought ...,✜°ɪ'ʟʟ ᴡᴀɪᴛ ғᴏʀ ʏᴏᴜ ʏᴏᴏɴɢɪ,BamgyuMin,ᵐⁱˢˢⁱⁿᵍ ᵗᵃᵏⁱᴇᴠʀʏᴅʏ.\n❣︎❣︎❣︎𖠌,523,2494,en,69,,1326736402831175680,Twitter for Android,https://twitter.com/BamgyuMin/status/132673640...
6,2020-11-12 03:59:10,RT @baejinnuna: Previously Hyunsuk mention in ...,an || ˣ¹ Reboot kajja!,myloveunsang,Multistan intinya mah saya teh suka banyak gru...,204,1331,en,48,🧡💙,1326736265962688512,Twitter for Android,https://twitter.com/myloveunsang/status/132673...
7,2020-11-12 03:58:38,RT @gizbot: Samsung Galaxy F41 Is Ready To Rul...,Gizbot Telugu,GizbotTelugu,"మొబైల్ లు ,లాప్ టాప్ లు ,టాబ్లెట్ ,సోషల్ నెట్వ...",698,15,en,4,,1326736134349742082,Twitter Web App,https://twitter.com/GizbotTelugu/status/132673...
8,2020-11-12 03:58:18,RT @yangwoning0209: #JUNGWON jungwonie bought ...,Sall 🐑 || 🌾,98_sbn,Day -19 🌄 || You and I We Can Fly 🕊..............,562,683,en,69,,1326736050425786370,Twitter for Android,https://twitter.com/98_sbn/status/132673605042...
9,2020-11-12 03:56:50,RT @yangwoning0209: #JUNGWON jungwonie bought ...,ENGENE 🐈,sheepinmoon,ENHYPEN ❤️ ENGENE\n.\n7 or Nothing,108,166,en,69,🇮🇩,1326735681771614210,Twitter for Android,https://twitter.com/sheepinmoon/status/1326735...


In [18]:
df.to_csv(r'C:\Users\Srinath Yasoda\JOBIFY\ ClassNotes\NLP\Output_Backup.csv',index=False)


In [75]:
df_fineTune=pd.read_csv('C:\\Users\\Srinath Yasoda\\JOBIFY\\ ClassNotes\\NLP\\Output.csv')
df_ft=df_fineTune[['Date','Text','Sentiment']]
print(df_ft.head())
print(df_ft.dtypes)


               Date                                               Text  \
0  12-11-2020 04:01  RT @SuperSaf: NEW VIDEO: iPhone 12 Pro Max vs ...   
1  12-11-2020 04:00  RT @SamsungMobile: Make their year, with Galax...   
2  12-11-2020 04:00  RT @baejinnuna: Previously Hyunsuk mention in ...   
3  12-11-2020 04:00  RT @yangwoning0209: #JUNGWON jungwonie bought ...   
4  12-11-2020 04:00  RT @baejinnuna: Previously Hyunsuk mention in ...   

  Sentiment  
0   Neutral  
1  Positive  
2  Positive  
3   Neutral  
4  Positive  
Date         object
Text         object
Sentiment    object
dtype: object


In [148]:
#Pickling the data
#df_ft.to_pickle('C:\\Users\\Srinath Yasoda\\JOBIFY\\ ClassNotes\\NLP\\df_bckup.pkl')

In [76]:
def Cleanthedata(text):
    text=(' '.join(re.sub("(RT @[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(text)).split()))   
    return text
    

In [77]:
import re
df_ft['Text']=df_ft['Text'].apply(Cleanthedata)


<ipython-input-77-f28da7cd1902>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ft['Text']=df_ft['Text'].apply(Cleanthedata)


In [81]:
print(df_ft.shape)
df_ft.head()

(1031, 3)


,Date,Text,Sentiment
0,12-11-2020 04:01,NEW VIDEO iPhone 12 Pro Max vs Samsung Galaxy ...,Neutral
1,12-11-2020 04:00,Make their year with Galaxy Buds Live Learn more,Positive
2,12-11-2020 04:00,Previously Hyunsuk mention in his previous Vli...,Positive
3,12-11-2020 04:00,JUNGWON jungwonie bought new phone it s samsun...,Neutral
4,12-11-2020 04:00,Previously Hyunsuk mention in his previous Vli...,Positive


In [147]:
df_ft['Sentiment'].value_counts()

Positive    180
Spam        172
Neutral     127
Negative     49
Neutal        1
 Spam         1
 Neutral      1
Name: Sentiment, dtype: int64

In [89]:
df_ft=df_ft.dropna()
df_ft.isna().count()

Date         531
Text         531
Sentiment    531
dtype: int64

In [130]:

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token=RegexpTokenizer(r'[a-zA-Z0-9]+')
cv=CountVectorizer(stop_words='english',ngram_range=(1,1),tokenizer=token.tokenize)
textcounts=cv.fit_transform(df_ft['Text'])

In [131]:
from sklearn.model_selection import train_test_split
X=textcounts
Y=df_ft['Sentiment']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=5)

In [133]:
#Bernouli--Features takes 2 values 0,1
#Multinomial--Discrete data, eg:movie ratings, text data analysis(most repeated word)
#Guassian--Because of assumptions of normal distributions GNB is used for continuous data
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
MNB=MultinomialNB()
MNB.fit(X_train,Y_train)
predict=MNB.predict(X_test)
MAccuracy=accuracy_score(Y_test,predict)
print('MNB accuracy score is {:04.2f}'.format(MAccuracy*100)+'%')

MNB accuracy score is 59.81%


## How to pickle and unpickle the data

In [183]:

#We use PICKLE to Serialize machine learning algorithms
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

MNB=MultinomialNB()
MNB.fit(X_train,Y_train)

#Saving the model using Pickle and Joblib

#1) Using PICKLE
filename='FinalizedModel.pkl'
pickle.dump(MNB,open(filename,'wb'))

#Load the pickled model
loadmodel=pickle.load(open(filename,'rb'))
result=loadmodel.score(X_test,Y_test)
print(result)

#2) Using Joblib
FN='FinalisedModelJoblib.pkl'
joblib.dump(MNB,FN)

Load=joblib.load(FN)
res=Load.score(X_test,Y_test)
print(res)

0.6074766355140186
0.6074766355140186


In [159]:
## Using (2,2) ngrams
cv=CountVectorizer(stop_words="english",ngram_range=(2,2),tokenizer=token.tokenize)
textcounts=cv.fit_transform(df_ft['Text'])
X=textcounts
Y=df_ft['Sentiment']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=5)
MNB=MultinomialNB()
MNB.fit(X_train,Y_train)
predict=MNB.predict(X_test)
MAccuracy2=accuracy_score(predict,Y_test)
print('MNB2 accuracy score is {:04.2f}'.format(MAccuracy2*100)+'%')
=--------------------------
## Using (3,3) ngrams
cv=CountVectorizer(stop_words="english",ngram_range=(3,3),tokenizer=token.tokenize)
textcounts=cv.fit_transform(df_ft['Text'])
X=textcounts
Y=df_ft['Sentiment']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=5)
MNB=MultinomialNB()
MNB.fit(X_train,Y_train)
predict=MNB.predict(X_test)
MAccuracy3=accuracy_score(predict,Y_test)
print('MNB3 accuracy score is {:04.2f}'.format(MAccuracy3*100)+'%')


MNB2 accuracy score is 60.75%


MNB3 accuracy score is 60.75%


## Using (2,2) NGRAMS applying all Naive Bayes algorithms

In [136]:
from sklearn.naive_bayes import ComplementNB
#from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

In [158]:
cv=CountVectorizer(stop_words="english",ngram_range=(2,2),tokenizer=token.tokenize)
textcounts=cv.fit_transform(df_ft['Text'])
X=textcounts
Y=df_ft['Sentiment']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=5)

#Bernoulli NB
BNB = BernoulliNB()
BNB.fit(X_train, Y_train)
accuracy_score_bnb = accuracy_score(BNB.predict(X_test),Y_test)
print('BNB accuracy = ' + str('{:4.2f}'.format(accuracy_score_bnb*100))+'%')


#Complement NB
CNB=ComplementNB()
CNB.fit(X_train,Y_train)
predict=CNB.predict(X_test)
CNB_Accuracy_score=accuracy_score(predict,Y_test)
print('CNB accuracy score is {:04.2f}'.format(CNB_Accuracy_score*100)+'%')


BNB accuracy = 62.62%
CNB accuracy score is 63.55%


## TermFrequencyInverseDocumentFrequency

In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer

In [141]:
tfidf=TfidfVectorizer()
texttoken2=tfidf.fit_transform(df_ft['Text'])

X=texttoken2
Y=df_ft['Sentiment']
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=5)


In [142]:
#MNB 
MNB.fit(x_train,y_train)
predict=MNB.predict(x_test)
MNB_AccuracyScore=accuracy_score(predict,y_test)
print('MNB accuracy score tfidf is {:04.2f}'.format(MNB_AccuracyScore*100)+'%')

#CNB
CNB.fit(x_train,y_train)
predict=CNB.predict(x_test)
CNB_Accuracy_score=accuracy_score(predict,y_test)
print('CNB accuracy score is {:04.2f}'.format(CNB_Accuracy_score*100)+'%')

#BNB
BNB.fit(x_train, y_train)
accuracy_score_bnb = accuracy_score(BNB.predict(x_test),y_test)
print('BNB accuracy = ' + str('{:4.2f}'.format(accuracy_score_bnb*100))+'%')

MNB accuracy score tfidf is 62.62%
CNB accuracy score is 63.55%
BNB accuracy = 62.62%


## *---Complement Naive Bayes gave good accuracy for both CountVectorizer and Tfidf Vectorizer---*

## Topic Modelling-- Latent Dirichlet Allocation(LDA) In Python

In [194]:
import pandas as pd
dflda1=pd.read_csv('C:\\Users\\Srinath Yasoda\\JOBIFY\\ ClassNotes\\NLP\\Output.csv')
dflda=dflda1[['Date','Text','Sentiment']]


In [198]:
dflda=dflda.dropna()

In [199]:
def Cleanthedata(text):
    text=(' '.join(re.sub("(RT @[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(text)).split()))   
    return text 
    

(531, 3)

In [200]:
dflda['Text']=dflda['Text'].apply(Cleanthedata)

In [202]:
dflda.head()

,Date,Text,Sentiment
0,12-11-2020 04:01,NEW VIDEO iPhone 12 Pro Max vs Samsung Galaxy ...,Neutral
1,12-11-2020 04:00,Make their year with Galaxy Buds Live Learn more,Positive
2,12-11-2020 04:00,Previously Hyunsuk mention in his previous Vli...,Positive
3,12-11-2020 04:00,JUNGWON jungwonie bought new phone it s samsun...,Neutral
4,12-11-2020 04:00,Previously Hyunsuk mention in his previous Vli...,Positive


In [203]:
data_text=dflda[['Text']]
data_text['Index']=data_text.index

<ipython-input-203-b16ee0e4fd43>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['Index']=data_text.index


In [204]:
data_text.head()

,Text,Index
0,NEW VIDEO iPhone 12 Pro Max vs Samsung Galaxy ...,0
1,Make their year with Galaxy Buds Live Learn more,1
2,Previously Hyunsuk mention in his previous Vli...,2
3,JUNGWON jungwonie bought new phone it s samsun...,3
4,Previously Hyunsuk mention in his previous Vli...,4


In [207]:
# Data Preprocessing
# !pip install gensim
import gensim
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS #for stopwords
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2020)

In [208]:
#!pip install gensim

In [211]:
#nltk.download('wordnet')


In [213]:
#Lemmatizing the words
print(WordNetLemmatizer().lemmatize('went',pos='v'))

#Stemmer example
stemmer=SnowballStemmer('english')
original_words=['caressessflies','mules','denied','agreed']

abc=[stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data={'Original word':original_words,'Stemmed':abc})


go


,Original word,Stemmed
0,caresses,caress
1,flies,fli
2,mules,mule
3,denied,deni
4,agreed,agre


In [215]:
#Defining Stemming and Lemmatizing functions
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(lemmatize_stemming(token))
    
    return result
            

In [221]:
doc=data_text
print(data_text[data_text['Index']==531])

                                                  Text  Index
531  Samsung Galaxy A01 A015M 16GB Dual SIM GSM Unl...    531


In [222]:
doc_sample=doc[doc['Index']==531].values[0][0]
print('Original Document:')
words=[]
for word in doc_sample.split(' '):
    
    words.append(word)
print(words)
print("\n\n tokenized and lemmatized document")
print(preprocess(doc_sample))

Original Document:
['Samsung', 'Galaxy', 'A01', 'A015M', '16GB', 'Dual', 'SIM', 'GSM', 'Unlocked', '5', '7', 'Display', 'Smartphone', 'International', 'Version', 'Red']


 tokenized and lemmatized document
['samsung', 'galaxi', 'dual', 'unlock', 'display', 'smartphon', 'intern', 'version']


In [223]:
processed_docs=doc['Text'].map(preprocess)

In [224]:
processed_docs[:10]

0    [video, iphon, samsung, galaxi, note, ultra, s...
1                     [year, galaxi, bud, live, learn]
2    [previous, hyunsuk, mention, previous, vlive, ...
3    [jungwon, jungwoni, buy, phone, samsung, galax...
4    [previous, hyunsuk, mention, previous, vlive, ...
5    [jungwon, jungwoni, buy, phone, samsung, galax...
6    [previous, hyunsuk, mention, previous, vlive, ...
7    [samsung, galaxi, readi, rule, segment, notch,...
8    [jungwon, jungwoni, buy, phone, samsung, galax...
9    [jungwon, jungwoni, buy, phone, samsung, galax...
Name: Text, dtype: object

## Bag of words on the dataset

In [227]:
dictionary=gensim.corpora.Dictionary(processed_docs)

In [233]:
count=0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>10:
        break

0 camera
1 comparison
2 galaxi
3 iphon
4 note
5 samsung
6 supersafstyl
7 ultra
8 video
9 bud
10 learn


In [234]:
dictionary.filter_extremes(no_below=15,no_above=0.5,keep_n=1000)


In [235]:
bow_corpus=[dictionary.doc2bow(item) for item in processed_docs]

In [243]:
bow_corpus[530]

[(21, 1)]

In [244]:
bow_corpus_530=bow_corpus[530]

for i in range(len(bow_corpus_530)):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus_530[i][0],dictionary[bow_corpus_530[i][0]],bow_corpus_530[i][1]))

Word 21 ("smartphon") appears 1 time.


## TFIDF

In [246]:
from gensim import corpora, models
tfidf=models.TfidfModel(bow_corpus)


In [247]:
corpus_tfidf=tfidf[bow_corpus]


In [248]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.8553554162330351), (1, 0.5180416121517762)]


## Running LDA using Bag of Words

In [249]:
lda_model=gensim.models.LdaMulticore(bow_corpus,num_topics=10,id2word=dictionary,passes=2,workers=2)

In [250]:
for idx,topic in lda_model.print_topics(-1):
    print('Topic:{} \nWords: {}'.format(idx,topic))

Topic:0 
Words: 0.426*"smartphon" + 0.178*"oneplus" + 0.110*"mobil" + 0.092*"appl" + 0.089*"realm" + 0.053*"phone" + 0.035*"check" + 0.004*"case" + 0.001*"year" + 0.001*"member"
Topic:1 
Words: 0.192*"phone" + 0.073*"note" + 0.073*"jungwon" + 0.073*"member" + 0.073*"enhypen" + 0.073*"jungwoni" + 0.073*"buy" + 0.066*"smartphon" + 0.058*"iphon" + 0.037*"oneplus"
Topic:2 
Words: 0.143*"phone" + 0.141*"buy" + 0.141*"jungwoni" + 0.141*"enhypen" + 0.141*"jungwon" + 0.141*"note" + 0.141*"member" + 0.001*"iphon" + 0.001*"previous" + 0.001*"case"
Topic:3 
Words: 0.364*"iphon" + 0.204*"phone" + 0.056*"oneplus" + 0.044*"case" + 0.044*"compat" + 0.044*"note" + 0.037*"previous" + 0.025*"appl" + 0.019*"better" + 0.019*"hyunsuk"
Topic:4 
Words: 0.163*"phone" + 0.132*"note" + 0.132*"enhypen" + 0.132*"jungwon" + 0.132*"member" + 0.132*"buy" + 0.132*"jungwoni" + 0.024*"case" + 0.004*"previous" + 0.003*"check"
Topic:5 
Words: 0.130*"phone" + 0.121*"note" + 0.121*"buy" + 0.121*"jungwoni" + 0.121*"member" 

## Running LDA using TF-IDF

In [252]:
lda_model_tfidf=gensim.models.LdaMulticore(corpus_tfidf,num_topics=10,id2word=dictionary,passes=2,workers=4)

In [253]:
for idx,topic in lda_model_tfidf.print_topics(-1):
    print("Topic:{} \nWords:{}".format(idx,topic))

Topic:0 
Words:0.141*"case" + 0.113*"previous" + 0.108*"iphon" + 0.066*"phone" + 0.060*"yesterday" + 0.060*"hyunsuk" + 0.060*"better" + 0.060*"vlive" + 0.060*"mention" + 0.026*"smartphon"
Topic:1 
Words:0.141*"jungwoni" + 0.141*"enhypen" + 0.141*"buy" + 0.141*"member" + 0.141*"jungwon" + 0.137*"note" + 0.112*"phone" + 0.008*"live" + 0.008*"learn" + 0.008*"year"
Topic:2 
Words:0.251*"year" + 0.251*"learn" + 0.251*"live" + 0.194*"bud" + 0.009*"smartphon" + 0.006*"oneplus" + 0.006*"check" + 0.006*"appl" + 0.001*"devic" + 0.001*"sad"
Topic:3 
Words:0.080*"buy" + 0.080*"jungwon" + 0.080*"jungwoni" + 0.080*"member" + 0.080*"enhypen" + 0.077*"note" + 0.066*"phone" + 0.037*"compat" + 0.033*"bud" + 0.031*"iphon"
Topic:4 
Words:0.215*"year" + 0.215*"live" + 0.215*"learn" + 0.172*"bud" + 0.014*"devic" + 0.014*"malaysia" + 0.014*"sad" + 0.014*"photocard" + 0.014*"compat" + 0.013*"check"
Topic:5 
Words:0.074*"phone" + 0.070*"note" + 0.067*"jungwon" + 0.067*"member" + 0.067*"enhypen" + 0.067*"jungwo

## Classification of the topics
## 1.Performance evalution by classifying sample document using LDA Bag of words model

In [255]:
processed_docs[530]

['jungwon',
 'jungwoni',
 'buy',
 'phone',
 'samsung',
 'galaxi',
 'note',
 'enhypen',
 'member']

In [257]:
for index,score in sorted(lda_model[bow_corpus[530]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score,lda_model.print_topic(index,10)))
    


Score: 0.5499924421310425	 
Topic: 0.426*"smartphon" + 0.178*"oneplus" + 0.110*"mobil" + 0.092*"appl" + 0.089*"realm" + 0.053*"phone" + 0.035*"check" + 0.004*"case" + 0.001*"year" + 0.001*"member"

Score: 0.0500035434961319	 
Topic: 0.192*"phone" + 0.073*"note" + 0.073*"jungwon" + 0.073*"member" + 0.073*"enhypen" + 0.073*"jungwoni" + 0.073*"buy" + 0.066*"smartphon" + 0.058*"iphon" + 0.037*"oneplus"

Score: 0.050003532320261	 
Topic: 0.130*"phone" + 0.121*"note" + 0.121*"buy" + 0.121*"jungwoni" + 0.121*"member" + 0.121*"jungwon" + 0.121*"enhypen" + 0.066*"smartphon" + 0.020*"oneplus" + 0.015*"mobil"

Score: 0.05000017583370209	 
Topic: 0.134*"phone" + 0.129*"member" + 0.129*"jungwoni" + 0.129*"note" + 0.129*"jungwon" + 0.129*"enhypen" + 0.129*"buy" + 0.012*"oneplus" + 0.009*"check" + 0.006*"bud"

Score: 0.05000017583370209	 
Topic: 0.242*"bud" + 0.236*"live" + 0.236*"learn" + 0.236*"year" + 0.006*"check" + 0.006*"compat" + 0.006*"malaysia" + 0.006*"sad" + 0.006*"photocard" + 0.006*"dev

## 2.Performance evalution by classifying document using LDA TFIDF Model

In [259]:
for index,score in sorted(lda_model_tfidf[bow_corpus[530]],key=lambda tup:-1*tup[1]):
    print("\nScore:{}\t \nTopic:{}".format(score,lda_model_tfidf.print_topic(index,10)))


Score:0.5499959588050842	 
Topic:0.250*"smartphon" + 0.131*"oneplus" + 0.100*"appl" + 0.097*"check" + 0.071*"mobil" + 0.049*"realm" + 0.037*"iphon" + 0.027*"phone" + 0.022*"photocard" + 0.022*"sad"

Score:0.05000196769833565	 
Topic:0.141*"case" + 0.113*"previous" + 0.108*"iphon" + 0.066*"phone" + 0.060*"yesterday" + 0.060*"hyunsuk" + 0.060*"better" + 0.060*"vlive" + 0.060*"mention" + 0.026*"smartphon"

Score:0.050001151859760284	 
Topic:0.149*"iphon" + 0.084*"year" + 0.084*"live" + 0.084*"learn" + 0.080*"note" + 0.077*"oneplus" + 0.067*"bud" + 0.047*"phone" + 0.042*"member" + 0.042*"buy"

Score:0.0500006377696991	 
Topic:0.251*"year" + 0.251*"learn" + 0.251*"live" + 0.194*"bud" + 0.009*"smartphon" + 0.006*"oneplus" + 0.006*"check" + 0.006*"appl" + 0.001*"devic" + 0.001*"sad"

Score:0.05000009387731552	 
Topic:0.141*"jungwoni" + 0.141*"enhypen" + 0.141*"buy" + 0.141*"member" + 0.141*"jungwon" + 0.137*"note" + 0.112*"phone" + 0.008*"live" + 0.008*"learn" + 0.008*"year"

Score:0.0500000

## Testing model on unseen data

In [260]:
unseen_doc='Previously Hyunsuk mention in his previous comment as Make their year with Galaxy Buds Live Learn more'
bow_vector=dictionary.doc2bow(preprocess(unseen_doc))

for index,score in sorted(lda_model[bow_vector],key=lambda tup:-1*tup[-1]):
    print("\nScore: {}\t Topic:{}".format(score,lda_model.print_topic(index,5)))


Score: 0.45601701736450195	 Topic:0.246*"previous" + 0.130*"iphon" + 0.117*"yesterday" + 0.117*"mention" + 0.117*"hyunsuk"

Score: 0.45509272813796997	 Topic:0.242*"bud" + 0.236*"live" + 0.236*"learn" + 0.236*"year" + 0.006*"check"

Score: 0.011111713945865631	 Topic:0.137*"bud" + 0.129*"check" + 0.119*"compat" + 0.119*"photocard" + 0.119*"devic"

Score: 0.011111676692962646	 Topic:0.364*"iphon" + 0.204*"phone" + 0.056*"oneplus" + 0.044*"case" + 0.044*"compat"

Score: 0.011111223138868809	 Topic:0.192*"phone" + 0.073*"note" + 0.073*"jungwon" + 0.073*"member" + 0.073*"enhypen"

Score: 0.011111140251159668	 Topic:0.163*"phone" + 0.132*"note" + 0.132*"enhypen" + 0.132*"jungwon" + 0.132*"member"

Score: 0.011111137457191944	 Topic:0.134*"phone" + 0.129*"member" + 0.129*"jungwoni" + 0.129*"note" + 0.129*"jungwon"

Score: 0.011111132800579071	 Topic:0.143*"phone" + 0.141*"buy" + 0.141*"jungwoni" + 0.141*"enhypen" + 0.141*"jungwon"

Score: 0.011111126281321049	 Topic:0.426*"smartphon" + 0.17

In [ ]:
## clustering out topics